# Wrapper function to create and setup a WKBeam run

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from wrapper_aux import *

## Specify the paths



In [2]:
parent_folder   = '/home/devlamin/WKbeam_simulations'

shot            = 86148
time            = 0.6
fluct_option    = 'nofluct_pext'

foldername = parent_folder + '/TCV_{:d}_{:.1f}_{}'.format(shot, time, fluct_option)

# Create the folder and inputfolders
subfolders = ['input', 'output', 'plots', 'configs']

folder = create_dir(foldername, subfolders)


Created directory: /home/devlamin/WKbeam_simulations/TCV_86148_0.6_nofluct_pext
Created directory: /home/devlamin/WKbeam_simulations/TCV_86148_0.6_nofluct_pext/input
Created directory: /home/devlamin/WKbeam_simulations/TCV_86148_0.6_nofluct_pext/output
Created directory: /home/devlamin/WKbeam_simulations/TCV_86148_0.6_nofluct_pext/plots
Created directory: /home/devlamin/WKbeam_simulations/TCV_86148_0.6_nofluct_pext/configs


At this point, the configs and input folders should be filled up first. Then continue notebook execution!

In [3]:
# Update the config files, for each of the configfiles found in the updated_configs folder
standard_configs = '/home/devlamin/WKBacca_QL/Wrapper/StandardConfigs'
updated_configs = '/home/devlamin/WKbeam_simulations/TCV_86148_0.6_nofluct_pext/configs'

for config in os.listdir(updated_configs):
    file_update = updated_configs + '/' + config
    
    file_standard = standard_configs + '/' + config.split('_')[0] + '.txt'
    file_output = foldername + '/' + config.split('_')[0] + '.txt'

    update_config(file_standard, file_update, file_output)
    print('Created config file: {}'.format(file_output))

Created config file: /home/devlamin/WKbeam_simulations/TCV_86148_0.6_nofluct_pext/Absorption.txt
Created config file: /home/devlamin/WKbeam_simulations/TCV_86148_0.6_nofluct_pext/AbsorptionUni.txt
Created config file: /home/devlamin/WKbeam_simulations/TCV_86148_0.6_nofluct_pext/Angular.txt
Created config file: /home/devlamin/WKbeam_simulations/TCV_86148_0.6_nofluct_pext/QLdiff.txt
Created config file: /home/devlamin/WKbeam_simulations/TCV_86148_0.6_nofluct_pext/RayTracing.txt
Created config file: /home/devlamin/WKbeam_simulations/TCV_86148_0.6_nofluct_pext/RhoThetaN.txt
Created config file: /home/devlamin/WKbeam_simulations/TCV_86148_0.6_nofluct_pext/XZ.txt


## Create executeable bash script

In [4]:

script_path = '/home/devlamin/WKBacca_QL/TCV_{:d}_{:.1f}_{}'.format(shot, time, fluct_option) + '.sh'
N_cores = 32

script_content = f"""#!/bin/bash
command1={foldername}/RayTracing.txt
mpiexec -np {N_cores} python3 WKBeam.py trace $command1
wait
python3 WKBeam.py bin {foldername}/Angular.txt
python3 WKBeam.py bin {foldername}/Absorption.txt
python3 WKBeam.py bin {foldername}/AbsorptionUni.txt
python3 WKBeam.py bin {foldername}/XZ.txt
python3 WKBeam.py bin {foldername}/RhoThetaN.txt
wait

echo "Ray tracing and binning done!"
mpiexec -np 22 python3 WKBeam.py QLdiff {foldername}/QLdiff.txt

echo " Quasilinear diffusion calculation done!"
"""

# Write to file
with open(script_path, "w") as f:
    f.write(script_content)

# Make executable
import os
os.chmod(script_path, 0o755)

print(f"Bash script created: {script_path}")

script_path = '/home/devlamin/WKBacca_QL/TCV_{:d}_{:.1f}_{}_plot.sh'.format(shot, time, fluct_option)

script_content = f"""#!/bin/bash
command1={foldername}/RayTracing.txt
python3 WKBeam.py plot2d {foldername}/Angular.txt &
python3 WKBeam.py plotabs {foldername}/Absorption.txt &
python3 WKBeam.py plotbin {foldername}/output/XZ_binned.hdf5 $command1 &
python3 WKBeam.py beamFluct {foldername}/output/XZ_binned.hdf5 $command1 &
wait
echo "All done!"
"""

# Write to file
with open(script_path, "w") as f:
    f.write(script_content)

# Make executable
import os
os.chmod(script_path, 0o755)

print(f"Bash script created: {script_path}")



Bash script created: /home/devlamin/WKBacca_QL/TCV_86148_0.6_nofluct_pext.sh
Bash script created: /home/devlamin/WKBacca_QL/TCV_86148_0.6_nofluct_pext_plot.sh
